In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [7]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob='*.pdf',
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [8]:
extracted_data = load_pdf("data/")

In [9]:
len(extracted_data)

637

In [10]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,
                                   chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
len(text_chunks)

7020

In [12]:
def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [13]:
embeddings = download_huggingface_embeddings()

In [14]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [15]:
query_result = embeddings.embed_query("Hello World")
len(query_result)

384

In [16]:
persist_directory = 'db'
vectordb = Chroma.from_documents(documents = text_chunks,
                                 embedding = embeddings,
                                 persist_directory = persist_directory)

In [17]:
vectordb.persist()

In [18]:
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embeddings)

In [19]:
retriever = vectordb.as_retriever(search_kwargs={"k":2})

In [20]:
docs = retriever.get_relevant_documents("What is allergy")
docs

[Document(page_content='When thisoccurs, an allergy develops against the offending sub-stance (an allergen.)', metadata={'page': 128, 'source': 'data\\Medical_book.pdf'}),
 Document(page_content='Purpose\nAllergy is a reaction of the immune system. Nor-', metadata={'page': 135, 'source': 'data\\Medical_book.pdf'})]

In [21]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [22]:
PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=['context', 'question'])
chain_type_kwargs = {"prompt": PROMPT}

In [23]:
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type='llama',
                    config={'max_new_tokens': 512,
                    'temperature': 0.8})

In [24]:
qa_chain = RetrievalQA.from_chain_type(llm = llm,
                                       chain_type='stuff',
                                       retriever=retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs=chain_type_kwargs)

In [25]:
while True:
    user_input = input(f"Ask the question: ")
    result = qa_chain({"query": user_input})
    print("Response: ", result['result'])
    

Response:  The most similar color to "out of direct light" is #666666.
Response:  Acne is a skin disease characterized by pimples on the face, chest, and back that occur when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
